# Cómo hacer una búsqueda a través del API de Twitter 

### Paso 1: Autenticación

En primer lugar dirígete a https://developer.twitter.com/en/apps y haz click en los **detalles de la aplicación** que has creado previamente. 

Vamos a crear el trozo de código que nos permitirá autenticarnos en el API de Twiter. Para facilitar la tarea, vamos a utilizar un *wrapper* 
Ahí te aparecerán 4 claves que necesitaremos que incluyas en el siguiente código respetando las comillas simples:

In [1]:
import tweepy
import csv

###incluye aqui tus claves unicas de Twitter
consumer_key = ''
consumer_secret= ''
access_token= '' 
access_token_secret= ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


### Paso 2: Crear fichero CSV

El siguiente paso será crear el fichero CSV donde vamos a almacenar los tweets que nos devuelva la plataforma. 

Escoge un nombre para el fichero, el que quieras.

Recuerda que el modo 'a' nos sirve para que al abrir repetidas veces el fichero, los contenidos nuevos se añadan y no reemplacen los existentes. Esto nos permite, por ejemplo, hacer repetidas búsquedas utilizando la misma consulta y almacenar los datos resultantes en el mismo fichero de forma incremental.

In [6]:
nombrefichero='tweets_3nov.csv'
csvFile = open(nombrefichero, 'a')
csvWriter = csv.writer(csvFile)

### Paso 3: Crear la consulta

¡Ha llegado el momento de montar nuestra primera consulta a la API de Twitter!

Te recomendamos que abras este enlace. Será útil para configurar consultas más complejas, filtrando por ejemplo tweets por usuario, o filtrar los dirigidos a un usuario en concreto o aquellos enviados desde una localización específica. 

https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators

La variable **query** será la que contenga las palabras clave o hashtags o nombres de usuario que queremos buscar en Twitter. Si incluimos *-filter:retweets* evitaremos que aparezcan tweets con el mismo texto en nuestro dataset. 

**Añade también el número de tweets que te gustaría obtener y el idioma en el que están escritos.** 

In [4]:
query="#voteearly -filter:retweets" 
number_of_tweets= 100
language="en" #es, en

### Paso 4: Procesar los resultados de la búsqueda

Cuando lanzamos una consulta a la API de Twitter, la aplicación nos devuelve un JSON similar al que se describe en este enlace:
https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets.html

Contendrá campos obligatorios y opcionales. Generalmente los obligatorios se encuentran al nivel de la raíz y los opcionales dentro de los campos más complejos como *entities*, *user* o *place*. 

```
**statuses** >> nombre del objeto raíz
    "created_at": "Mon May 06 20:01:29 +0000 2019",
    "id": 1125490788736032770,
    "id_str": "1125490788736032770",
    "text": "Today's new update means that you can finally add Pizza Cat to your Retweet with comments! Learn more about this ne… https://t.co/Rbc9TF2s5X",
    "truncated": true,
    *"entities"*: {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [],
        "urls": []
        }
    *"user":*
    *"place":*
    
    ...
```    
En este caso vamos a centrarnos en capturar la fecha, el nombre del usuario, la localización y el tweet; algunos campos no suelen aparecer en los tweets a no ser que la geolocalización esté habilitada en el tweet del propio usuario.

Recordad que utilizaremos el encoding "utf-8" para ver correctamente los caracteres y especialmente los emojis. Python 3 lo incorpora por defecto. 



**Añade al dataset el nombre del usuario que ha escrito el tweet**

*Pista: busca por screen_name en* https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
csvWriter.writerow(["Created_at", "User_name", "Location", "Tweet", "Retweets"])

for tweet in tweepy.Cursor(api.search,q=query,
                           count=number_of_tweets,
                           tweet_mode="extended",
                           lang=language,
                           since="2020-11-01").items():
    
        print(tweet.created_at , tweet.user.screen_name,  tweet.user.location, tweet.full_text, tweet.retweet_count)
        csvWriter.writerow([tweet.created_at, tweet.user.screen_name, tweet.user.location, tweet.full_text, tweet.retweet_count])  
    
  #      Python 2.7
  #      print (tweet.created_at, tweet.user.location.encode('utf-8'), tweet.text.encode('utf-8'), tweet.coordinates, tweet.place.country)
  #      csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.coordinates, tweet.place.country])

    
       
    
    

2020-11-03 10:50:50 DemetreBhunter California USA  https://t.co/UA4SEuWOoU  #joebiden  #Electionday2020  #vote2020  #VoteEarly #VoteEarlyDay 🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 win of ultra amazing super of winner 0
2020-11-03 10:47:43 DemetreBhunter California USA  https://t.co/nEZ1kDKN6c  #joebiden #electionday #election2020 #vote #vote2020  he going to win today for  #electionday2020  #VoteEarly #VoteEarlyDay 🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 0
2020-11-03 10:46:44 Ramsesgust Nicaragua libre somewhere @Reuters Fact:👇👇👇👇
corruption lies to the nation, Illicit enrichment "Joe Biden" 
https://t.co/vcQPznlTGF
#VOTE
#VoteEarlyDay
#VoteEarlyFL
#VoteEarly #ElectionDay  #Election2020 2
2020-11-03 10:46:31 Ramsesgust Nicaragua libre somewhere @thehill Fact:👇👇👇👇
corruption lies to the nation, Illicit enrichment "Joe Biden" 
https://t.co/vcQPznlTGF
#VOTE
#VoteEarlyDay
#VoteEarlyFL
#VoteEarly #ElectionDay  #Election2020 0
2020-11-03 10:44:55 Ramsesgust Nicaragua libre somewhere @MSNBC Fact:👇👇👇👇
corruption lies to the natio

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files

files.download('__________.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Revisa ahora tu fichero csv y analiza los tweets que has obtenido